# Question:

## What is the gross annual produced hydrocarbons on Norwegian Continental Shelf through time?

In [ ]:
import numpy as np
import pandas as pd
import janitor

In [ ]:
production_wellbores_monthly =  pd.read_csv("https://factpages.npd.no/ReportServer_npdpublic?/FactPages/TableView/field_production_gross_monthly&rs:Command=Render&rc:Toolbar=false&rc:Parameters=f&rs:Format=CSV&Top100=false&IpAddress=82.102.27.204&CultureCode=en")
production_wellbores_monthly.head()

In [ ]:
production_wellbores_monthly.columns = production_wellbores_monthly.columns.str.replace("prf", "")
production_wellbores_monthly = production_wellbores_monthly.clean_names(case_type="snake")

In [ ]:
production_wellbores_monthly_LONG = (production_wellbores_monthly
    .drop(columns=["information_carrier", "npdid_information_carrier", "prd_oe_gross_mill_sm3"])
    .melt(
        id_vars=["year", "month"],
        var_name="hc_phase",
        value_name="prd_gross_mill_sm3"
        )
    .groupby(["year", "hc_phase"])
    .agg(prd_gross_mill_sm3_year = pd.NamedAgg(column="prd_gross_mill_sm3", aggfunc="sum"))
    .reset_index()
    ).sort_values(["year", "hc_phase"])

In [ ]:
production_wellbores_monthly_LONG.head()

In [ ]:
import altair as alt
alt.data_transformers.disable_max_rows() # default is 5000 rows warning
#alt.renderers.enable('notebook')

domain = ['prd_condensate_gross_mill_sm3', 'prd_gas_gross_bill_sm3',
       'prd_oil_gross_mill_sm3', 'prd_produced_water_in_field_mill_sm3']
range_ = ['pink', 'red', 'green', 'blue']

annual_production_plot = (alt
    .Chart(production_wellbores_monthly_LONG)
    .mark_bar(opacity=0.3)
    .encode(
        x="year:Q",
        y="prd_gross_mill_sm3_year:Q",
        color=alt.Color('hc_phase', scale=alt.Scale(domain=domain, range=range_)),
        row="hc_phase:N"
        )
    .properties(height=100)
)

annual_production_plot

In [ ]:
(annual_production_plot
    .encode(tooltip=['prd_gross_mill_sm3_year', 'year', 'hc_phase'])
    .interactive())